# Brasil documents

https://link.lens.org/Kr7ZrPzLlJf: Downloaded JSON files [here](https://drive.google.com/drive/folders/1tnLxHd4Jz2s35OdFlnqXRFv4iUEqZ76h?usp=sharing)

[Google Scholar](http://scholar.google.com/scholar_lookup?&title=WIMP+dark+matter+as+radiative+neutrino+mass+messenger&journal=JHEP&volume=10&publication_year=2013&author=Hirsch,M&author=Lineros,RA&author=Morisi,S&author=Palacio,J&author=Rojas,N&author=Valle,JWF') example 

## Prepare data collection

### With  Pandas
Requieres enough RAM

Get the lens.org JSON files

In [1]:
import pandas as pd

In [2]:
br=pd.read_excel('/scratch/restrepo/Publicacoes-periodicos_2013-2020-Scopus.xlsx')

In [3]:
['Art. No.','CODEN','PubMed ID']

['Art. No.', 'CODEN', 'PubMed ID']

In [4]:
br[~br['PubMed ID'].isna()].empty

True

In [5]:
nanc=[c for c in br.columns
    if br[~br[c].isna()].empty]

In [6]:
br=br.drop(nanc,axis='columns')

In [7]:
br.to_excel('Publicacoes-periodicos_2013-2020-Scopus.xlsx',index=False)

In [8]:
br=pd.read_excel('Publicacoes-periodicos_2013-2020-Scopus.xlsx')

In [9]:
br['ID bolsistas']=br['ID bolsistas'].str.replace("'","")
br.columns

Index(['Authors', 'Title', 'Year', 'Source title', 'Volume', 'Issue',
       'Page start', 'Page end', 'DOI', 'Author Keywords', 'ISSN',
       'ID bolsistas'],
      dtype='object')

In [10]:
br['pages']=br.apply(lambda row: f"{row['Page start']}-{row['Page end']}" if isinstance(row['Page end'],int) else row['Page start'] ,axis='columns')

In [11]:
from IPython.display import  JSON

JSON Scheme:

In [12]:
JSON( {'title': 'Asociación Herpetológica Argentina: Treinta años de esfuerzos y resultados',
 'author': 'Williams, JD',
 'journal': 'Cuadernos de Herpetología',
 'year': 2013,
 'volume': '27',
 'issue': '1',
 'doi': '',
 'publisher': '',
 'country': '',
 'pages': '',
 'abstract': '',
 'article_id': '',
 'language': ''})

<IPython.core.display.JSON object>

In [13]:
br=br.rename({'Title':'title',
              'Authors':'author',
              'Source title':'journal',
              'Year':'year',
              'Volume':'volume',
              'Issue':'issue',
              'DOI':'doi',
             },axis='columns')

In [14]:
br['publisher']=''
br['country']=''
br["abstract"]=''
br["language"]=''
br["article_id"]=br["article_id"]=br.index

In [15]:
br.to_json('brasil.json', orient='records') #,orient='records'

In [16]:
ls -lh brasil.json

-rw-r--r-- 1 colav colav 322M Nov 12  2023 brasil.json


See [sample.ipynb]('./sample.ipynb') for installation instrucctions of the Google Scholar downloader tool, `moai`

connect to mongodb host, and (Default port is 27017). Optiona use of other port is report
```bash
$ cd
$ mongod --dbpath data/db/ [--port 27018]
```
and cp `brasil.json` there. Then

```bash
mongoimport --db br --collection data --file /scratch/restrepo/brasil.json --jsonArray [--port 27018]
hunabku_server [--db_port 27018]
moai_gslookup --hunabku_server http://10.0.0.8:8080 --proxy_api XXXXX --max_threads 50 --db br --max_tries 2
```
<!--XXX →  12894ec7c83f91ffcdff8fb116e75906 -->

Monitor the running of `moai_gslookup`
```bash
$ mongo [--port 27018]
> use br
switched to db br
> show collections
data
> show collections
data
quarantine
stage
> db.stage.count()
372
```

### Creates de chache
```bash
moai_gscites  --hunabku_server http://10.0.0.8:8080 --proxy_apikey xxxxxx --db br --create_cache

```

### Download metadata info for each cite
Creates 
```bash
moai_gscites  --hunabku_server http://10.0.0.8:8080 --proxy_apikey xxxxxx --db br --max_threads 50  --max_tries 2
```

## Dump

```bash
cd /scratch/restrepo/dumps/br/
mongodump -d br [--port 27018]
#To restore in a new mongo without the la database:
mongorestore ./dump/
```

backup 
```bash
local$ rsync -avxz dump colav@clustercien.udea.edu.co:covid
local$ ssh colav@clustercien.udea.edu.co
password:
clustercien$ cd covid
clustercien$ ls
dump
clustercien$ mongorestore ./dump/
```

## Check the API

In [17]:
import requests
import getpass

In [ ]:
api=getpass.getpass('API key')

In [3]:
rr=requests.get(f"http://api.scraperapi.com/account?api_key={api}")

In [4]:
rr.json()

{'concurrencyLimit': 50,
 'concurrentRequests': 0,
 'failedRequestCount': 187934,
 'requestCount': 3000225,
 'requestLimit': '3000000'}

### With Mongodb
Only disc

In [4]:
from pymongo import MongoClient
from bson.objectid import ObjectId
import sys

In [5]:
client = MongoClient()

In [6]:
client = MongoClient('172.19.31.8', 27017)

In [7]:
#client.drop_database('la')

In [8]:
client.list_database_names()

['MA',
 'argentina',
 'argentina_new_scholar',
 'argentina_tmp',
 'books',
 'br',
 'colciencias-mabooks',
 'colciencias-names',
 'colciencias-test',
 'colciencias-test-bk',
 'colciencias-test_books',
 'colombia',
 'colombia_impactu',
 'colombia_udea',
 'covidbsm',
 'inspirehep',
 'la',
 'local',
 'mm_books',
 'scholar_colombia_update',
 'scholar_colombia_update_2022',
 'scienti_2021',
 'sds',
 'sds_bk_07112022',
 'sds_old',
 'test-book-single',
 'test-book-single-parse']

In [7]:
db = client['br']

In [8]:
db.list_collection_names()

['stage_cites',
 'stage_cites_invalid',
 'quarantine',
 'stage_invalid',
 'cache_cites_count',
 'error_cites_count',
 'gslookup_not_found',
 'data',
 'cache_cites',
 'stage']

In [11]:
dt=db.get_collection('data')

In [12]:
st=db.get_collection('stage')

In [13]:
st.count_documents({})

418177

In [12]:
i=20201
l=[d for d in st.find({'article_id':str(i)})]

In [14]:
from IPython.display import JSON

In [28]:
d=l[0]

True

In [45]:
import bson

In [ ]:
bson.ObjectId

In [43]:
type(d['_id'])

bson.objectid.ObjectId

In [46]:
type(d['_id'])==bson.objectid.ObjectId

True

In [50]:
JSON( [ dict(zip(d.keys(),[str(x) if isinstance(d['_id'],bson.objectid.ObjectId) else x for x in d.values()]))] )

<IPython.core.display.JSON object>

Count citations

In [13]:
import time

In [23]:
s=time.time()
n=0
i=0
for d in st.find():
    nc=d.get('cites')
    if nc:
        n=n+nc
    
    if i%10000==0:
        print(str(n).zfill(7),end='\r')
    i=i+1
print('')
print(time.time()-s)

6678492
5.3602800369262695


In [24]:
n

6740871

## Update

In [49]:
ct=pd.read_excel('/scratch/restrepo/citacoes.xlsx')

In [50]:
ct.shape

(437952, 16)

In [51]:
ct.columns

Index(['ID', 'author', 'title', 'year', 'journal', 'volume', 'issue',
       'Page start', 'Page end', 'doi', 'Author Keywords', 'ISSN',
       'ID bolsistas', 'pages', 'cites', 'Coautorías'],
      dtype='object')

In [53]:
ct[:3]

,ID,author,title,year,journal,volume,issue,Page start,Page end,doi,Author Keywords,ISSN,ID bolsistas,pages,cites,Coautorías
0,1,"SANCHES R., RAZKY A.",Variação Linguística para Cigarro de Palha e T...,2015,Revista Todas as Letras (MACKENZIE. Online),17,NaN,196,206,10.15529/1980-6914/letras.v17n3p196-206,NaN,19806914,8153913927369006,196-206,0,2
1,2,"RAZKY A., SANCHES R.D.",VARIAÇÃO GEOSSOCIAL DO ITEM LEXICAL ?RIACHO/CÓ...,2016,GRAGOATÁ,21,NaN,70,89,10.22409/gragoata.v21i40.33375,Dialetologia Pluridimensional; Variação lexica...,23584114,8153913927369006,70-89,8,2
2,3,"OLIVEIRA M.B.D., LIMA A.F.d., RAZKY A.",Palatalização de l diante de i no português br...,2016,Revista Linguística (Online),32,NaN,1,10,10.5935/2079-312x.20160017,ATLAS LINGUÍSTICO DO BRASIL; geografia lingüís...,2079312X,8153913927369006,1-10,1,3


In [54]:
br[:3]

,Authors,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,Page end,...,CODEN,PubMed ID,Language of Original Document,Abbreviated Source Title,Document Type,Publication Stage,Access Type,Source,EID,ID bolsistas
0,"SANCHES R., RAZKY A.",NaN,Variação Linguística para Cigarro de Palha e T...,2015,Revista Todas as Letras (MACKENZIE. Online),17,NaN,NaN,196,206,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'8153913927369006
1,"RAZKY A., SANCHES R.D.",NaN,VARIAÇÃO GEOSSOCIAL DO ITEM LEXICAL ?RIACHO/CÓ...,2016,GRAGOATÁ,21,NaN,NaN,70,89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'8153913927369006
2,"OLIVEIRA M.B.D., LIMA A.F.d., RAZKY A.",NaN,Palatalização de l diante de i no português br...,2016,Revista Linguística (Online),32,NaN,NaN,1,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'8153913927369006


In [14]:
dt=pd.DataFrame(list(dt.find()))

In [15]:
dt.columns

Index(['_id', 'author', 'title', 'year', 'journal', 'volume', 'issue',
       'Page start', 'Page end', 'doi', 'Author Keywords', 'ISSN',
       'ID bolsistas', 'pages', 'publisher', 'country', 'abstract', 'language',
       'article_id'],
      dtype='object')

In [16]:
dt.iloc[0]

_id                                         6120755f7786ebe168225119
author                                          SANCHES R., RAZKY A.
title              Variação Linguística para Cigarro de Palha e T...
year                                                            2015
journal                  Revista Todas as Letras (MACKENZIE. Online)
volume                                                            17
issue                                                           None
Page start                                                       196
Page end                                                         206
doi                          10.15529/1980-6914/letras.v17n3p196-206
Author Keywords                                                 None
ISSN                                                        19806914
ID bolsistas                                        8153913927369006
pages                                                        196-206
publisher                         

In [17]:
dt.shape

(437952, 19)

In [18]:
st=pd.DataFrame(list(st.find()))

In [19]:
st.shape

(418177, 24)

In [55]:
st['GS_title']=st['title']
st['updated_cites']=st['cites']

In [56]:
up=ct.merge(st[['_id', 'GS_title',
        'updated_cites', 'cites_link',  'profiles',  'bibtex', 'old_title', 'cid']],
               left_on='title',right_on='old_title',how='left')#.dropna(subset=['Title']).shape

In [57]:
ct.shape

(437952, 16)

In [61]:
up.columns

Index(['ID', 'author', 'title', 'year', 'journal', 'volume', 'issue',
       'Page start', 'Page end', 'doi', 'Author Keywords', 'ISSN',
       'ID bolsistas', 'pages', 'cites', 'Coautorías', '_id', 'GS_title',
       'updated_cites', 'cites_link', 'profiles', 'bibtex', 'old_title',
       'cid'],
      dtype='object')

In [70]:
up=up.drop_duplicates(subset=['ID']).reset_index(drop=True)

In [71]:
up[:3]

,ID,author,title,year,journal,volume,issue,Page start,Page end,doi,...,cites,Coautorías,_id,GS_title,updated_cites,cites_link,profiles,bibtex,old_title,cid
0,1,"SANCHES R., RAZKY A.",Variação Linguística para Cigarro de Palha e T...,2015,Revista Todas as Letras (MACKENZIE. Online),17,NaN,196,206,10.15529/1980-6914/letras.v17n3p196-206,...,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,"RAZKY A., SANCHES R.D.",VARIAÇÃO GEOSSOCIAL DO ITEM LEXICAL ?RIACHO/CÓ...,2016,GRAGOATÁ,21,NaN,70,89,10.22409/gragoata.v21i40.33375,...,8,2,6120755f7786ebe16822511a,Variação geossocial do item lexical �riacho/có...,7.0,/scholar?cites=4228313119509545427&as_sdt=2005...,"{'A Razky': 'IMgC-vAAAAAJ', 'RD Sanches': 'QTy...","@article{razky2016variaccao,\n title={Varia{\...",VARIAÇÃO GEOSSOCIAL DO ITEM LEXICAL ?RIACHO/CÓ...,04F9L2H8rToJ
2,3,"OLIVEIRA M.B.D., LIMA A.F.d., RAZKY A.",Palatalização de l diante de i no português br...,2016,Revista Linguística (Online),32,NaN,1,10,10.5935/2079-312x.20160017,...,1,3,6120755f7786ebe16822511b,Palatalização de L diante de I no português br...,3.0,"/scholar?cites=373025853626654700&as_sdt=5,33&...","{'MB Oliveira': 'tstjx6cAAAAJ', 'A Razky': 'IM...","@article{oliveira2016palatalizaccao,\n title=...",Palatalização de l diante de i no português br...,7B90mQlBLQUJ


In [76]:
up['updated_cites']=up['updated_cites'].fillna(0).astype(int)

In [78]:
up=up.fillna('')

In [80]:
up.to_excel('citacoes_updated.xlsx',index=False)

In [20]:
br.columns

Index(['Authors', 'Author(s) ID', 'Title', 'Year', 'Source title', 'Volume',
       'Issue', 'Art. No.', 'Page start', 'Page end', 'Page count', 'Cited by',
       'DOI', 'Link', 'Affiliations', 'Authors with affiliations', 'Abstract',
       'Author Keywords', 'Index Keywords', 'Molecular Sequence Numbers',
       'Chemicals/CAS', 'Tradenames', 'Manufacturers', 'Funding Details',
       'Funding Text 1', 'Funding Text 2', 'Funding Text 3', 'Funding Text 4',
       'Funding Text 5', 'References', 'Correspondence Address', 'Editors',
       'Sponsors', 'Publisher', 'Conference name', 'Conference date',
       'Conference location', 'Conference code', 'ISSN', 'ISBN', 'CODEN',
       'PubMed ID', 'Language of Original Document',
       'Abbreviated Source Title', 'Document Type', 'Publication Stage',
       'Access Type', 'Source', 'EID', 'ID bolsistas'],
      dtype='object')

In [21]:
st.columns

Index(['_id', 'journal', 'publisher', 'country', 'article_id', 'title',
       'author', 'doi', 'year', 'volume', 'issue', 'pages', 'language',
       'abstract', 'cites', 'cites_link', 'ref', 'profiles', 'pdf', 'cid',
       'rp', 'bibtex', 'old_title', 'timestamp'],
      dtype='object')

In [22]:
st['_id']=st['_id'].astype(str)
dt['_id']=dt['_id'].astype(str)

In [23]:
br.columns

Index(['Authors', 'Author(s) ID', 'Title', 'Year', 'Source title', 'Volume',
       'Issue', 'Art. No.', 'Page start', 'Page end', 'Page count', 'Cited by',
       'DOI', 'Link', 'Affiliations', 'Authors with affiliations', 'Abstract',
       'Author Keywords', 'Index Keywords', 'Molecular Sequence Numbers',
       'Chemicals/CAS', 'Tradenames', 'Manufacturers', 'Funding Details',
       'Funding Text 1', 'Funding Text 2', 'Funding Text 3', 'Funding Text 4',
       'Funding Text 5', 'References', 'Correspondence Address', 'Editors',
       'Sponsors', 'Publisher', 'Conference name', 'Conference date',
       'Conference location', 'Conference code', 'ISSN', 'ISBN', 'CODEN',
       'PubMed ID', 'Language of Original Document',
       'Abbreviated Source Title', 'Document Type', 'Publication Stage',
       'Access Type', 'Source', 'EID', 'ID bolsistas'],
      dtype='object')

In [24]:
st.columns

Index(['_id', 'journal', 'publisher', 'country', 'article_id', 'title',
       'author', 'doi', 'year', 'volume', 'issue', 'pages', 'language',
       'abstract', 'cites', 'cites_link', 'ref', 'profiles', 'pdf', 'cid',
       'rp', 'bibtex', 'old_title', 'timestamp'],
      dtype='object')

In [37]:
up=br.merge(st[['_id', 'title',
        'cites', 'cites_link',  'profiles',  'bibtex', 'old_title', 'cid']],
               left_on='Title',right_on='old_title',how='left')#.dropna(subset=['Title']).shape

In [38]:
up.dropna(subset=['old_title']).shape

(439507, 58)

In [39]:
up.dropna(subset=['Title']).shape

(485344, 58)

In [40]:
br.shape

(464361, 50)

In [ ]:
up=(br.sort_values('cites',ascending=False)).drop_duplicates(subset=['Authors', 'Author(s) ID', 'Title', 'Year',
       'Source title', 'Volume', 'Issue', 'Art. No.', 'Page start', 'Page end',
       'Page count', 'Cited by', 'DOI', 'Link', 'Affiliations',
       'Authors with affiliations', 'Abstract', 'Author Keywords',
       'Index Keywords', 'Molecular Sequence Numbers', 'Chemicals/CAS',
       'Tradenames', 'Manufacturers', 'Funding Details', 'Funding Text 1',
       'Funding Text 2', 'Funding Text 3', 'Funding Text 4', 'Funding Text 5',
       'References', 'Correspondence Address', 'Editors', 'Sponsors',
       'Publisher', 'Conference name', 'Conference date',
       'Conference location', 'Conference code', 'ISSN', 'ISBN', 'CODEN',
       'PubMed ID', 'Language of Original Document',
       'Abbreviated Source Title', 'Document Type', 'Publication Stage',
       'Access Type', 'Source', 'EID', 'ID bolsistas']).reset_index(drop=True)

In [ ]:
up.shape #464361

In [45]:
writer = pd.ExcelWriter('/scratch/colav/Publicacoes-periodicos_2013-2020-Scopus_updated.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'strings_to_urls': False}})

up.to_excel(writer,index=False)
writer.close()

In [48]:
up.cites.dropna().sum()

11009121.0

In [52]:
up.iloc[1].to_dict()

{'Authors': 'RAZKY A., SANCHES R.D.',
 'Author(s) ID': nan,
 'Title': 'VARIAÇÃO GEOSSOCIAL DO ITEM LEXICAL ?RIACHO/CÓRREGO? NAS CAPITAIS BRASILEIRAS',
 'Year': 2016,
 'Source title': 'GRAGOATÁ',
 'Volume': 21,
 'Issue': nan,
 'Art. No.': nan,
 'Page start': 70,
 'Page end': 89,
 'Page count': nan,
 'Cited by': nan,
 'DOI': '10.22409/gragoata.v21i40.33375',
 'Link': nan,
 'Affiliations': nan,
 'Authors with affiliations': nan,
 'Abstract': nan,
 'Author Keywords': 'Dialetologia Pluridimensional; Variação lexical; ATLAS LINGUÍSTICO DO BRASIL',
 'Index Keywords': nan,
 'Molecular Sequence Numbers': nan,
 'Chemicals/CAS': nan,
 'Tradenames': nan,
 'Manufacturers': nan,
 'Funding Details': nan,
 'Funding Text 1': nan,
 'Funding Text 2': nan,
 'Funding Text 3': nan,
 'Funding Text 4': nan,
 'Funding Text 5': nan,
 'References': nan,
 'Correspondence Address': nan,
 'Editors': nan,
 'Sponsors': nan,
 'Publisher': nan,
 'Conference name': nan,
 'Conference date': nan,
 'Conference location': n

# Appendix

In [80]:
REBUILD=False
if not REBUILD:
    dt=db.get_collection('data')
    sys.exit('Already loaded')
else:
    dt=db.get_collection('data')

for p in ln.find():
    dd={}
    l=[]
    for d in p['authors'][:3]:
        l.append(f'{d.get("last_name")}, {d.get("initials")}')
    dd['author']=' and '.join(l)
    if isinstance(p.get('source'),dict):
        dd['journal']=p['source'].get('title_full')
    else:
        dd['journal']=''
    for d in p['external_ids']:
        if d.get('type')=='doi':
            doi=d.get('value')
        else:
            doi=''
    dd['doi']=doi
    dd["pages"]=p.get('start_page')            

    dd['title']=p['title']
    dd['volume']=p.get('volumne')
    dd['issue']=p.get('issue')     
    
    dd['year']=p['year_published']                     
    dd['article_id']=p['lens_id']                     
    dd['publisher']=''
    dd['country']=''
    dd["abstract"]=''
    dd["language"]=''
    dt.insert_one(dd)
    #break

SystemExit: Already loaded

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [74]:
dt.count_documents({})

133853

In [75]:
#dt.drop()

### Add the lens collection `'_id'` to data as `'pid'` by searching with `'title'`

In [76]:
ln.count_documents({})

136326

In [77]:
dt.count_documents({})

133853

In [34]:
import time

In [81]:
REBUILD=False
if not REBUILD:
    sys.exit('Already fixed')
#See https://stackoverflow.com/a/24200795/2268280
cursor = ln.find(
     no_cursor_timeout=True
)

s=time.time()
for d in cursor:
    title=d.get('title')
    pid=str(d.get('_id'))
    if title and pid:
        kk=dt.update_one( {"title":title}, {"$set" : {"pid":pid}}   )
    #break
cursor.close()
print(time.time()-s)

SystemExit: Already fixed

In [82]:
i=0
iend=55
for d in dt.find():
    if i==iend:
        break
    i=i+1

In [83]:
d['pid'],d['title']

('60e63d00cdd7c50a878a7984',
 'Parasitism and physiological trade-offs in stressed capybaras.')

In [84]:
dd=list(ln.find({'_id':ObjectId(d['pid'])}))[0]

In [85]:
dd['title']

'Parasitism and physiological trade-offs in stressed capybaras.'

### Check de collection with Pandas

In [86]:
import pandas as pd

In [87]:
ar=pd.DataFrame(list(dt.find()))
ar[:3]

,_id,title,author,journal,year,volume,issue,doi,publisher,country,pages,abstract,article_id,language,pid
0,60d6957a70d175acd8cb1340,Low vitamin D concentrations among indigenous ...,"Hirschler, V and Maccallini, G and Molinari, C",Pediatric diabetes,2013,14,3,,,,,,,,60e63d00cdd7c50a878a794f
1,60d6957a70d175acd8cb1341,Investigaciones sobre género y determinación p...,"Tajer, D and Reid, G and Gaba, M",Revista Argentina de Cardiología,2013,81,4,10.7775/rac.es.v81.i4.280,,,,,,,60e63d00cdd7c50a878a7949
2,60d6957a70d175acd8cb1342,Duplex Stainless Steels - Low‐Cycle Fatigue at...,"Alvarez-Armas, I",Duplex Stainless Steels,2013,,,10.1002/9781118557990.ch6,,,,,,,60e63d00cdd7c50a878a7947


## Obtain the cites
`--proxy_api` key from https://dashboard.scraperapi.com/dashboard

```bash
hunabku_server #in and indepentdent, nohup, other terminal
```

### Get de Google Scholar info for each document
Including `'cid'` and bibtex. The info is stores in the stage collection

```bash
moai_gslookup --hunabku_server http://10.0.0.8:8080 --proxy_api XXXXXX --max_threads 50 --db la --max_tries 2
```

Check the proxies

In [89]:
st=db.get_collection('stage')
st.count_documents({})

125987

In [90]:
s=st.find_one()
cid=s['cid']
(s['title'],s['author'],s['journal'])

('Low vitamin D concentrations among indigenous Argentinean children living at high altitudes',
 'Hirschler, Valeria and Maccallini, Gustavo and Molinari, Claudia and Aranda, Claudio and San Antonio de los Cobres Study Group',
 'Pediatric diabetes')

Check with:

In [91]:
print( f'https://scholar.google.com/scholar?q=info:{cid}:scholar.google.com' )

https://scholar.google.com/scholar?q=info:bCwnnx4CbTsJ:scholar.google.com


In [92]:
ar=pd.DataFrame(list(st.find()))

In [93]:
ar.cites.fillna(0).sum()

1277822.0

### Creates de chache
```bash
moai_gscites  --hunabku_server http://127.0.1.1:8080 --proxy_apikey xxxxxx --db la --create_cache

```

### Download metadata info for each cite
Creates 
```bash
moai_gscites  --hunabku_server http://127.0.1.1:8080 --proxy_apikey xxxxxx --db la --max_threads 5  --max_tries 2
```

In [94]:
sc=db.get_collection('stage_cites')
sc.count_documents({})

1076343

In [147]:
sce=sc.find({'pid':str(s['_id'])})

In [148]:
ar=pd.DataFrame(list(sce))
ar[:1]

,_id,pid,cite
0,60dae3041d38555bfdc170a2,60d6957a70d175acd8cb1340,{'journal': 'American journal of hypertension'...


In [125]:
cites=s['cites_link'].split('cites=')[1].split('&')[0]

In [ ]:
ar=pd.DataFrame( ar['cite'].to_list())

To check with

In [126]:
f'https://scholar.google.com/scholar?cites={cites}'

'https://scholar.google.com/scholar?cites=4282081151241628780'

In [116]:
ar[['author','title','journal','bibtex']]

,author,title,journal,bibtex
0,"Tomaino, Katherine and Romero, Karina M and Ro...",Association between serum 25-hydroxy vitamin D...,American journal of hypertension,"@article{tomaino2015association,\n title={Ass..."
1,"Hirschler, Valeria and Maccallini, Gustavo and...",Improvement in high-density lipoprotein choles...,Hormone research in paediatrics,"@article{hirschler2013improvement,\n title={I..."
2,"Hirschler, V",Cardiometabolic risk factors in native populat...,International journal of clinical practice,"@article{hirschler2016cardiometabolic,\n titl..."
3,"Orces, Carlos H",Vitamin D status among older adults residing i...,The Scientific World Journal,"@article{orces2015vitamin,\n title={Vitamin D..."
4,"Farrokhyar, Forough and Tabasinejad, Rasam and...",Prevalence of vitamin D inadequacy in athletes...,Sports medicine,"@article{farrokhyar2015prevalence,\n title={P..."
5,"Kelishadi, Roya and Ardalan, Gelayol and Motla...",National report on the association of serum vi...,Nutrition,"@article{kelishadi2014national,\n title={Nati..."
6,"Skutsch, Margaret and Dobler, Carlos and McCal...",The association of UV with rates of COVID-19 t...,medRxiv,"@article{skutsch2020association,\n title={The..."
7,"Gassmann, Max and Mairbäurl, Heimo and Livshit...",The increase in hemoglobin concentration with ...,Annals of the New York Academy of Sciences,"@article{gassmann2019increase,\n title={The i..."
8,", and ,",D ...,. .,"@article{ 2016 ,\n title={ ..."
9,"OUASTI, Asmaa and AGUILI, Zeyneb",LStatut De LStatut De LStatut De La Vitamine D...,,"@phdthesis{ouastilstatut,\n title={LStatut De..."


Convert `ObjectId` to `str`

In [136]:
sid=str(s['_id'])

In [145]:
pd.DataFrame( list( st.find({'_id':ObjectId( sid )}) ) )[['article_id','title','author','journal','cites']]

,article_id,title,author,journal,cites
0,,Low vitamin D concentrations among indigenous ...,"Hirschler, Valeria and Maccallini, Gustavo and...",Pediatric diabetes,40


## Check the API

In [1]:
import requests
import getpass

In [2]:
api=getpass.getpass('API key')

API key ································


In [9]:
rr=requests.get(f"http://api.scraperapi.com/account?api_key={api}")

In [10]:
rr.json()

{'concurrencyLimit': 50,
 'concurrentRequests': 0,
 'failedRequestCount': 0,
 'requestCount': 0,
 'requestLimit': '3000000'}

## Dump

```bash
cd /scratch/restrepo/dumps/br/
mongodump -d br
#To restore in a new mongo without the la database:
mongorestore ./dump/
```

Google Scholar Profile
* cites can be obtained directly
* Click on title and in the open up change the "related:" by "info:"

## Quality check

In [181]:
ar=pd.DataFrame()
cursor = sc.find(
     no_cursor_timeout=True
)

s=time.time()
i=0
for d in cursor:
    ar=ar.append( {'title': d.get('cite').get('title'),'cid':d.get('cite').get('cid')},ignore_index=True )
    i+=1
    if i%10000==0:
        print(i,end='\r')
    #break
cursor.close()
print(time.time()-s)

26685.94394993782


In [190]:
ar.cid.dropna().shape

(1076343,)

In [188]:
ar.cid.unique().shape

(767437,)

In [186]:
ar.shape

(1076343, 2)

In [195]:
ar[ar.cid.apply(len)!=12].shape

(0, 2)

In [236]:
ar[ar['title'].str.strip().apply(len)<20]

,cid,title
17,ttOESFOc6AQJ,D ...
20,3iTk9qCfhqgJ,d ...
22,-Q5wI4PeXUIJ,Neurotropic Viruses
122,E6fJtJufdMgJ,D ...
144,3aIHrYiwd3kJ,
...,...,...
1076134,omR61rw4zC8J,. ...
1076136,E4cbAxkmM-sJ,ESCUELA DE POSGRADO
1076162,_Djbm4jBOD4J,CRx MAGAZINE
1076165,2GFieJ0Mfb0J,Food Bioscience


The failed ones seem to be in other languages

In [235]:
cid='3aIHrYiwd3kJ'
print( f'https://scholar.google.com/scholar?q=info:{cid}:scholar.google.com' )

https://scholar.google.com/scholar?q=info:3aIHrYiwd3kJ:scholar.google.com


In [16]:
import pandas as pd

In [19]:
pd.DataFrame([{'A':None}]).to_json('kk.json',orient='records')

In [20]:
cat kk.json

[{"A":null}]

In [23]:
df=pd.read_json('https://computationalmethods.github.io/Evaluacion_2021-1/calificaciones.json')

In [24]:
df

,student_id,Parcial 1,Parcial 2,Parcial 3,Trabajo Final,Tareas,%
0,0,"{'value': 0, '%': 15, 'description': 'Capítulo...","{'value': 0, '%': 20, 'description': 'Capítulo...","{'value': None, '%': 20, 'description': 'Capít...","{'value': None, '%': 15, 'description': 'Capít...","[{'Tarea': '1', 'value': 0, 'description': 'Py...",30
1,1113674432,"{'value': 0, '%': 15, 'description': 'Capítulo...","{'value': 0, '%': 20, 'description': 'Capítulo...","{'value': None, '%': 20, 'description': 'Capít...","{'value': None, '%': 15, 'description': 'Capít...","[{'Tarea': '1', 'value': 0, 'description': 'Py...",30
2,98554575,"{'value': 0, '%': 15, 'description': 'Capítulo...","{'value': 0, '%': 20, 'description': 'Capítulo...","{'value': None, '%': 20, 'description': 'Capít...","{'value': None, '%': 15, 'description': 'Capít...","[{'Tarea': '1', 'value': 0, 'description': 'Py...",30
